In [1]:
import tensorflow as tf

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import KFold 
from catboost import CatBoostClassifier

In [56]:
train = pd.read_csv('train.csv')
greeks = pd.read_csv('greeks.csv')
test = pd.read_csv('test.csv')
print(train.shape, test.shape)

(617, 58) (5, 57)


In [ ]:
#train['EJ'] = train['EJ'].replace({'A':0,'B':1})

In [57]:
def fill_empty(df):
    featuresWithNulls = df.columns[df.isnull().any()]
    imputer = KNNImputer(n_neighbors=5)
    df[featuresWithNulls] = imputer.fit_transform(df[featuresWithNulls])
    return df

train =fill_empty(train)

### VIF - Мультиколлинеарность (не используем)

In [ ]:
def check_vif(df):
    vifs = [variance_inflation_factor(df, i) for i in range(df.shape[1])]
    vif_df = pd.DataFrame({"features":df.columns, "VIF" : vifs})
    vif_df = vif_df.sort_values(by="VIF", ascending=False)
    remove_col = vif_df.iloc[0, 0]
    top_vif = vif_df.iloc[0, 1]
    return vif_df, remove_col, top_vif

top_vif = 100
while(top_vif > 5):
    vif_df, remove_col, top_vif = check_vif(train)
    print(remove_col, top_vif)
    if top_vif < 5:
        break
    train = train.drop(columns=remove_col)

### Метод главных компонент PCA  (не используем)

In [ ]:
X = train.drop(columns=["Class"])
y = train['Class']
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)
pca = PCA(n_components='mle') # автоматический выбор компонент
pca_result = pca.fit_transform(scaled_data)
pca_df = pd.DataFrame(data = pca_result,columns=[f"PC{i}" for i in range(1, pca_result.shape[1]+1)])

### Генерация фичей

In [45]:
def findMostCorrColumns(data, topk):
    topk += 1
    values = np.abs(train.drop("Id", axis = 1).corr().loc[:,'Class']).sort_values()[-topk:-1]
    columns = values.index
    return columns

def groupCalculate(data):
    columns = data.columns.drop('EJ')
    index = 0
    for column1 in columns:
        index += 1
        for column2 in columns[index:]:
            data['{}_{}_RATE'.format(column1,column2)] = data[column1] / data[column2]
    return data

In [58]:
TARGET = train.Class
train['EJ'] = train['EJ'].replace({'A':0,'B':1})

In [47]:
train = train[findMostCorrColumns(train, -1)]
train = groupCalculate(train)
train['Class'] = TARGET

C:\Users\Колонтай\AppData\Local\Temp\ipykernel_544\566209103.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['{}_{}_RATE'.format(column1,column2)] = data[column1] / data[column2]
C:\Users\Колонтай\AppData\Local\Temp\ipykernel_544\566209103.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['{}_{}_RATE'.format(column1,column2)] = data[column1] / data[column2]
C:\Users\Колонтай\AppData\Local\Temp\ipykernel_544\566209103.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calli

C:\Users\Колонтай\AppData\Local\Temp\ipykernel_544\566209103.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['{}_{}_RATE'.format(column1,column2)] = data[column1] / data[column2]
C:\Users\Колонтай\AppData\Local\Temp\ipykernel_544\566209103.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['{}_{}_RATE'.format(column1,column2)] = data[column1] / data[column2]
C:\Users\Колонтай\AppData\Local\Temp\ipykernel_544\566209103.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calli

C:\Users\Колонтай\AppData\Local\Temp\ipykernel_544\566209103.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['{}_{}_RATE'.format(column1,column2)] = data[column1] / data[column2]
C:\Users\Колонтай\AppData\Local\Temp\ipykernel_544\566209103.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['{}_{}_RATE'.format(column1,column2)] = data[column1] / data[column2]
C:\Users\Колонтай\AppData\Local\Temp\ipykernel_544\566209103.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calli

C:\Users\Колонтай\AppData\Local\Temp\ipykernel_544\566209103.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['{}_{}_RATE'.format(column1,column2)] = data[column1] / data[column2]
C:\Users\Колонтай\AppData\Local\Temp\ipykernel_544\566209103.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['{}_{}_RATE'.format(column1,column2)] = data[column1] / data[column2]
C:\Users\Колонтай\AppData\Local\Temp\ipykernel_544\566209103.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calli

### Обучение на CatBoost со стратификацией

In [49]:

oof = np.zeros((len(train), 2))
skf = KFold(n_splits=5, shuffle=True, random_state=42)

params={
    'iterations':10000,
    'learning_rate':0.005,
    'early_stopping_rounds':1000,
    'auto_class_weights':'Balanced',
    'loss_function':'MultiClass',
    'eval_metric':'MultiClass:use_weights=True',
    'random_seed':42,
    'use_best_model':True,
    'l2_leaf_reg':1,
    'max_ctr_complexity':15,
    'max_depth':10,
    "grow_policy":'Lossguide',
    'max_leaves':64,
    "min_data_in_leaf":40,
    }

In [50]:
for train_index,val_index in skf.split(train, greeks.iloc[:,1:-1]):

    train_x, valid_x = train.loc[train_index, train.columns.drop('Class')], train.loc[val_index, train.columns.drop('Class')]
    train_y, valid_y = train.loc[train_index, 'Class'], train.loc[val_index, 'Class']
    
    model = CatBoostClassifier(**params)
    model.fit(train_x,train_y,eval_set=[(valid_x,valid_y)], verbose=200, early_stopping_rounds=100)
    preds = model.predict_proba(valid_x)
    oof[val_index, :] = preds